# 4.1 compute the parity of a word
e.g. 

the parity of 1001 is 0(for it has two 1s) 

the parity of 1000 is 1(for it has one 1)

## Bitwise operator

Let x = 10 (0000 1010 in binary) and y = 4 (0000 0100 in binary)

|Operator| Meaning | Example
|---|---|---|
|& | Bitwise AND	| x & y = 0 (0000 0000)
|\| |	Bitwise OR	| x | y = 14 (0000 1110)
|~	| Bitwise NOT	| ~x = -11 (1111 0101)
|^	| Bitwise XOR	| x ^ y = 14 (0000 1110)
|\>\>	| Bitwise right shift |	x >> 2 = 2 (0000 0010)
|\<\<	| Bitwise left shift	|x << 2 = 40 (0010 1000)

* NOT
  ```
  NOT 0111  (decimal 7)
    = 1000  (decimal 8)
  ```
* AND
  ```
      0101 (decimal 5)
  AND 0011 (decimal 3)
    = 0001 (decimal 1)
  ```

* OR

  ```
    0010 (decimal 2)
  OR 1000 (decimal 8)
  = 1010 (decimal 10)
  ```

* XOR
  ```
      0101 (decimal 5)
  XOR 0011 (decimal 3)
    = 0110 (decimal 6)
  ```

In [15]:
# 4.1 brute force
def parity(x):
    result=0
    while x:
        result ^= x & 1 # result = result XOR (x & 1), if x !=0
        x >>= 1 # x=x(binary) right shift 1
    return result

#### e.g.
```
x=4 (bi: 100)
1: x=100(bi) ; result = 000 XOR (100 & 001) = 000 XOR 000 = 000
2: x=010(bi) ; result = 000 XOR (010 & 001) = 000 XOR 000 = 000
3: x=001(bi) ; result = 000 XOR (001 & 001) = 000 XOR 001 = 001
4: x=000(bi)
return result=1
```

```
x=5  (bi: 101)
1: x=101(bi) ; result = 000 XOR (101 & 001) = 000 XOR 001 = 001
2: x=010(bi) ; result = 001 XOR (010 & 001) = 001 XOR 000 = 001
3: x=001(bi) ; result = 001 XOR (001 & 001) = 001 XOR 001 = 000
4: x=000(bi)
return result=0
```

In [30]:
# 4.1 erasing the lowest set bit
def parity(x):
    result=0
    while x:
        result ^=  1 
        x &= x-1 # Drops the lowest set bit of x
    return result

In [ ]:
# 4.1 using 16 bit mask

# first make a dictionary which has info like below
# in the case 16 bits subwords, it may look simmilar
# like {0000 0000 0000 0000:0, 
#       0000 0000 0000 0001:1, ...}
PRECOMPUTED_PARITY={"base 16 bit subwords":"corresponding parity"}

def parity(x):
    MASK_SIZE=16

    # 0x is like a declaimer telling us this is going to be hexadecimal
    # F=15 in hexadecimal, in binary form F=1111
    # we have 4 Fs here, therefore, 0xFFFF= 1111 1111 1111 1111
    BIT_MASK=0xFFFF
    
    # for a 64-bit integer, it would have 4 16-bit subwords
    subword1_parity=PRECOMPUTED_PARITY[x>>(3 * MASK_SIZE)]
    subword2_parity=PRECOMPUTED_PARITY[x>>(2 * MASK_SIZE) & BIT_MASK]
    subword3_parity=PRECOMPUTED_PARITY[x>>(1 * MASK_SIZE) & BIT_MASK]
    subword4_parity=PRECOMPUTED_PARITY[x & BIT_MASK]
    
    return subword1_parity^subword2_parity^subword3_parity^subword4_parity

### bit-fiddling tips summary(1)
x=5(10**1**)
* clear the lowest bit in x
    ```
    x&(x-1)=101&100=100
    ```
* extract the lowest bit of x
    ```
    x& ~(x-1)=101&(~100)=101&011=001
    ```
* get the last two bits of x using mask `11`
    ```
        101
    AND 011
    =   001
    ```
* computing parity of a integer(110100) from parity of subwords(11,01,00)
    ```
    parity(11) = 0 , parity(01) = 1 , parity(00) = 0
    parity(11 01 00) =  0 ^ 1 ^ 0 = 1
    ```

# 4.2 Swap Bits
Given a 64-bit integer, swap the bit at indices i and j

### bit-fiddling tips summary(2)
* get the ith bit of x
    ```
    (x>>i)&1
    ```
* swaping using `11` mask
    ```
        10          01
    XOR 11      XOR 11
    =   01      =   10
    ```

In [1]:
# 4.2 brute-force
def swap_bits(x,i,j):
    if (x>>i)&1 != (x>>j)&1: # if the ith and jth bit differ
        bit_mask=(1<<i)|(1<<j) # bit_mask=0..010..010...0, where ith and jth bit is 1
        x^=bit_mask # x= x XOR bit_mask
    return x
        

# 4.3 Reverse Bits
* The order of bits reverse
    ```
    10011 -> 11001
    ```
* Reversing `10 01 00 11` is euqal to in turn reversing `rev(11)`, `rev(00)`, `rev(01)`, `rev(10)`

### bit-fiddling tips summary(3)
* any integer do `OR`  operation with `0` of same length equals the integer itself
    ```
    x=5(101)
    
       101
    OR 000
    =  101
    ```

In [9]:
# 4.3 16 bit mask based on premade lookup table
# similar to 4.1 using 16 bit mask
PRECOMPUTED_PARITY={"base 16 bit subwords":"reversed bits"}

def swap_bits(x):
    MASK_SIZE=16
    BIT_MASK=0xFFFF

    reverse1=PRECOMPUTED_PARITY[x                & BIT_MASK] << 3*MASK_SIZE # put the last 16 bits to the front
    reverse2=PRECOMPUTED_PARITY[(x>>  MASK_SIZE) & BIT_MASK] << 2*MASK_SIZE
    reverse3=PRECOMPUTED_PARITY[(x>>2*MASK_SIZE) & BIT_MASK] << 1*MASK_SIZE
    reverse4=PRECOMPUTED_PARITY[(x>>3*MASK_SIZE) & BIT_MASK]
    return reverse1|reverse2|reverse3|reverse4


0.12